<a href="https://colab.research.google.com/github/md-junaid79/MultiAgent-with-Autogen-/blob/main/Multiagent_system_using_autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install autogen

In [54]:
!pip install pyautogen

In [55]:
!pip install dask[dataframe]

In [56]:
import autogen

In [57]:
from autogen import AssistantAgent, UserProxyAgent

In [58]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [59]:
# config_list = [{'model': 'gemini-1.5-flash', 'api_key': "GOOGLE_API_KEY" ,"api_type": "google"}]

In [60]:
# llm_config={"config_list": config_list}

In [61]:
# Make sure you have your Google API key stored in Colab secrets under the name 'GOOGLE_API_KEY'
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

config_list = [
    {
        'model': 'gemini-2.5-flash',
        'api_key': GOOGLE_API_KEY,
        'api_type': 'google'
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0.3,              # You can adjust the temperature as needed
    "timeout": 120,                  # You can adjust the timeout as needed
}


In [62]:
gemini_flash_config = {
    "cache_seed": 42,      # change the cache_seed for different trials
    "temperature": 0.3,
    "config_list": config_list,
    "timeout": 120,
}


In [63]:
task = '''
 **Task**: As an software architect, you are required to design a solution for the
 following business requirements:
    - Data storage for massive amounts of IoT,customers and various data
    - Real-time data analytics and machine learning pipeline
    - Scalability
    - Cost Optimization
    - Timeline: 6 months

    Ensure that your solution architecture is following best practices.
    '''

In [64]:
cloud_prompt = '''
**Role**: You are an expert cloud architect. You need to develop architecture proposals
using either cloud-specific PaaS services, or cloud-agnostic ones.
The final proposal should consider all 3 main cloud providers: Azure, AWS and GCP, and provide
a data architecture for each. At the end, briefly state the advantages of cloud over on-premises
architectures, and summarize your solutions for each cloud provider using a table for clarity.
'''
cloud_prompt += task

In [65]:
oss_prompt = '''
**Role**: You are an expert of open-source software architecture. You need
to develop architecture proposals without considering cloud solutions.
 Only use open-source frameworks that are popular and have lots of active contributors.
 At the end, briefly state the advantages and summarize your solutions using a table for clarity.
'''
oss_prompt += task

In [66]:
lead_prompt =  '''
**Role**: You are a lead Architect tasked with managing a conversation between
the cloud and the open-source Architects.
Each Architect will perform a task and respond with their results. You will critically
review those and also ask for, or point to, the disadvantages of their solutions.
You will review each result with chain of thought approach and choose the best solution in accordance with the business
requirements and architecture best practices. You will use any number of summary tables to communicate your decision.
'''
lead_prompt += task

In [67]:
user_proxy=UserProxyAgent(
    name="supervisor",
    system_message = "A Human Head of Architecture",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="NEVER",


)

In [68]:
cloud_agent = autogen.AssistantAgent(
    name = "cloud",
    system_message = cloud_prompt,
    llm_config=llm_config
    )

oss_agent = autogen.AssistantAgent(
    name = "oss",
    system_message = oss_prompt,
    llm_config=llm_config
    )

lead_agent = autogen.AssistantAgent(
    name = "lead",
    system_message = lead_prompt,
    llm_config=llm_config
)

In [69]:
def state_transition(last_speaker,groupchat):
   messages = groupchat.messages

   if last_speaker is user_proxy:
       return cloud_agent
   elif last_speaker is cloud_agent:
       return oss_agent
   elif last_speaker is oss_agent:
       return lead_agent
   elif last_speaker is lead_agent:
       # lead -> end
       return None

supervisoragent-->cloudagent-->ossagent-->leadagent

In [70]:
groupchat=autogen.GroupChat(
    agents=[user_proxy, cloud_agent, oss_agent, lead_agent],
    messages=[],
    max_round=3,
    speaker_selection_method=state_transition,
)

In [71]:
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gemini_flash_config)

In [72]:
user_proxy.initiate_chat(
    manager, message="Provide your best architecture based on the AI agent for the procurement business requirements."
)

supervisor (to chat_manager):

Provide your best architecture based on the AI agent for the procurement business requirements.

--------------------------------------------------------------------------------

Next speaker: cloud

cloud (to chat_manager):

As an expert cloud architect, I've designed a robust, scalable, and cost-optimized solution for your procurement business, leveraging the power of cloud-native Platform-as-a-Service (PaaS) offerings. This approach prioritizes rapid development and deployment, making the 6-month timeline achievable while adhering to best practices in data architecture, security, and operations.

The core architecture follows a modern data lakehouse pattern, combining the flexibility and cost-effectiveness of a data lake with the structure and performance of a data warehouse. This enables both real-time analytics and sophisticated machine learning pipelines on massive datasets.

---

## **Core Architectural Principles & Best Practices**

1.  **Serverle

ChatResult(chat_id=52740268824588307620662016663688523832, chat_history=[{'content': 'Provide your best architecture based on the AI agent for the procurement business requirements.', 'role': 'assistant', 'name': 'supervisor'}, {'content': "As an expert cloud architect, I've designed a robust, scalable, and cost-optimized solution for your procurement business, leveraging the power of cloud-native Platform-as-a-Service (PaaS) offerings. This approach prioritizes rapid development and deployment, making the 6-month timeline achievable while adhering to best practices in data architecture, security, and operations.\n\nThe core architecture follows a modern data lakehouse pattern, combining the flexibility and cost-effectiveness of a data lake with the structure and performance of a data warehouse. This enables both real-time analytics and sophisticated machine learning pipelines on massive datasets.\n\n---\n\n## **Core Architectural Principles & Best Practices**\n\n1.  **Serverless First